In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

# Import API key
from api_keys import api_key
single_city_url = "http://api.openweathermap.org/data/2.5/weather?"

## Bokeh imports

In [6]:
#%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import DatetimeTickFormatter
##import pandas as pd
#import numpy as np
from bokeh.models.tools import HoverTool
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
city_name = "portland"
city_id = "5746545"
city_units = "metric"
covid = "New_York_Times/covid-19-state-level-data.csv"
covid_transport = "transport_cases_table.csv"

In [12]:
covid_df = pd.read_csv(covid_transport)

In [16]:
#covid_df
covid_df.rename(columns = {'throw':'local_index'}, inplace = True)

In [19]:
covid_df = covid_df.drop('index_row',axis=1)

## data cleaning

In [20]:
covid_df

,local_index,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,0,2020-02-15,Washington,53,1,0,0,0,United States,2.0,-1.0,8.0,8.0,-3.0,0.0
1,1,2020-02-16,Washington,53,1,0,0,0,United States,15.0,3.0,19.0,5.0,-1.0,-1.0
2,2,2020-02-17,Washington,53,1,0,0,0,United States,12.0,2.0,63.0,-7.0,-35.0,6.0
3,3,2020-02-18,Washington,53,1,0,0,0,United States,5.0,2.0,52.0,7.0,-4.0,0.0
4,4,2020-02-19,Washington,53,1,0,0,0,United States,4.0,3.0,60.0,7.0,-2.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,1635,2020-03-31,West Virginia,54,162,1,17,0,United States,-37.0,-15.0,-14.0,-28.0,-42.0,15.0
1636,1636,2020-04-01,West Virginia,54,191,2,29,1,United States,-34.0,-10.0,-14.0,-26.0,-43.0,15.0
1637,1637,2020-04-02,West Virginia,54,217,2,26,0,United States,-35.0,-10.0,-2.0,-31.0,-42.0,16.0
1638,1638,2020-04-03,West Virginia,54,242,2,25,0,United States,-35.0,-10.0,8.0,-36.0,-42.0,17.0


In [21]:
covcopy = covid_df.copy()

In [22]:
state_list = covid_df['state'].unique()

In [23]:
state_list

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'West Virginia'], dtype=object)

## Isolate regions by state

In [24]:
region_list_country = covid_df.loc[covid_df['state'] == "Oregon"]

In [25]:
region_list_country.head(5)

,local_index,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
438,438,2020-02-28,Oregon,41,1,0,1,0,United States,8.0,4.0,24.0,7.0,3.0,-1.0
439,439,2020-02-29,Oregon,41,1,0,0,0,United States,7.0,5.0,8.0,6.0,3.0,-1.0
440,440,2020-03-01,Oregon,41,2,0,1,0,United States,17.0,9.0,23.0,9.0,3.0,-1.0
441,441,2020-03-02,Oregon,41,2,0,0,0,United States,8.0,7.0,14.0,2.0,3.0,0.0
442,442,2020-03-03,Oregon,41,2,0,0,0,United States,10.0,12.0,40.0,2.0,1.0,-1.0


In [26]:
cases_state = [case for case in region_list_country['cases']]

In [27]:
new_case = []
for day in range(0,len(cases_state)):
    if day == 0:
        new_case.append(cases_state[day])
    elif cases_state[day] > cases_state[day - 1]:
        new_case.append(cases_state[day] - cases_state[day - 1])
    else:
        new_case.append(0)

In [28]:
region_list_country['new_cases'] = new_case

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
deaths_state = [case for case in region_list_country['deaths']]

In [30]:
new_deaths = []
for day in range(0,len(deaths_state)):
    if day == 0:
        new_deaths.append(deaths_state[day])
    elif deaths_state[day] > deaths_state[day - 1]:
        new_deaths.append(deaths_state[day] - deaths_state[day - 1])
    else:
        new_deaths.append(0)

In [31]:
region_list_country['new_deaths'] = new_deaths

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
region_list_country.head(5)

,local_index,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
438,438,2020-02-28,Oregon,41,1,0,1,0,United States,8.0,4.0,24.0,7.0,3.0,-1.0
439,439,2020-02-29,Oregon,41,1,0,0,0,United States,7.0,5.0,8.0,6.0,3.0,-1.0
440,440,2020-03-01,Oregon,41,2,0,1,0,United States,17.0,9.0,23.0,9.0,3.0,-1.0
441,441,2020-03-02,Oregon,41,2,0,0,0,United States,8.0,7.0,14.0,2.0,3.0,0.0
442,442,2020-03-03,Oregon,41,2,0,0,0,United States,10.0,12.0,40.0,2.0,1.0,-1.0


In [33]:
ref_date = []

for date in region_list_country['date']:
    ref_date.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

date_groups = []

for date in indy_location_moving[0]['date']:
    date_groups_1.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

In [34]:
date_cases = []

for date in ref_date:
    date_cases.append(str(date))

# pass plot data as well as hovertool data in to plot.
source_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['cases'],
            desc=date_cases,
        )
    )

In [35]:
# pass plot data as well as hovertool data in to plot.
source_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['deaths'],
            desc=date_cases,
        )
    )

In [36]:
# pass plot data as well as hovertool data in to plot.
source_new_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['new_deaths'],
            desc=date_cases,
        )
    )

In [37]:
# pass plot data as well as hovertool data in to plot.
source_new_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['new_cases'],
            desc=date_cases,
        )
    )

In [47]:
# pass plot data as well as hovertool data in to plot.
source_retail = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['retail'],
            desc=date_cases,
        )
    )

In [38]:
# configure plot tools including the hover tool / tooltips.  Displays date / time of individual data points when hovering 
#over individual points.
TOOLSCCD = [HoverTool(tooltips=[
            ("cvd_date", "@desc"),("cases","$y")],
                  formatters={'DateTime': 'datetime'}),'wheel_zoom','save','crosshair','pan','reset']

In [39]:
cid = figure(title = f"Values for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cid.xaxis.axis_label = 'Event Date'
cid.yaxis.axis_label = 'COVID Total Cases'
# Setup x axis for using Datetime formatting.
cid.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cid.xaxis.major_label_orientation = pi/4
cid.circle('x', 'y', line_color="navy", fill_color="red", fill_alpha=0.2, size=10,source=source_cases)


GlyphRenderer(id='1046', ...)

In [40]:
cix = figure(title = f"Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cix.xaxis.axis_label = 'Event Date'
cix.yaxis.axis_label = 'COVID Total Deaths'
# Setup x axis for using Datetime formatting.
cix.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cix.xaxis.major_label_orientation = pi/4
cix.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_deaths)


GlyphRenderer(id='1088', ...)

In [41]:
cin = figure(title = f"New Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cin.xaxis.axis_label = 'Event Date'
cin.yaxis.axis_label = 'COVID New Deaths'
# Setup x axis for using Datetime formatting.
cin.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cin.xaxis.major_label_orientation = pi/4
cin.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_new_deaths)

GlyphRenderer(id='1130', ...)

In [42]:
cinc = figure(title = f"New Cases for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cinc.xaxis.axis_label = 'Event Date'
cinc.yaxis.axis_label = 'COVID New Cases'
# Setup x axis for using Datetime formatting.
cinc.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cinc.xaxis.major_label_orientation = pi/4
cinc.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_new_cases)

GlyphRenderer(id='1172', ...)

In [48]:
cinr = figure(title = f"Retail movement for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cinr.xaxis.axis_label = 'Event Date'
cinr.yaxis.axis_label = 'Retail activity'
# Setup x axis for using Datetime formatting.
cinr.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cinr.xaxis.major_label_orientation = pi/4
cinr.circle('x', 'y', line_color="navy", fill_color="green", fill_alpha=0.2, size=10,source=source_retail)

GlyphRenderer(id='1827', ...)

In [43]:
show(cid)

In [44]:
show(cix)

In [45]:
show(cin)

In [46]:
show(cinc)

In [49]:
show(cinr)